In [1]:
pwd

'C:\\Projects\\Structures\\scripts'

In [6]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

sys.path.append('../')
#from IPM_Shared_Code_public.Python.geo_functions import read_geosql
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *
from get_address_points import *

In [7]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [8]:
geo_key = config['keys']['geosupport_key']
geo_ip = config['keys']['geosupport_ip']

In [9]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                  ';Database=' + structuresdb +
                                  ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

In [6]:
#b = pd.read_csv('structs_top5.csv')
#b

In [10]:
#sql = 'select top 15 * from structuresdb.dbo.tbl_parks_structures'
sql = 'select top 15 * from structuresdb.dbo.tbl_parks_structures where bin = ''1027194'''

In [11]:
b = pd.read_sql(con=engine, sql=sql)

In [19]:
b

,fid,objectid,parks_id,bin,bbl,doitt_id,ground_elevation,height_roof,construction_year,alteration_year,demolition_year,n_doitt_ids,doitt_source,overlap_except,unexpected_change,parks_overlap,shape,row_hash
0,991,5383,MT03-BLG0001,1027194,1011130051,313850,None,None,1924,None,None,1,None,True,False,False,b'\xd7\x08\x00\x00\x01\x04\x13\x00\x00\x00\x00...,b'\xfa\xe6\xfd+\xbe\x08\xfe\xff\\\xd8\x99\xd0\...


In [18]:
b.apply(lambda x: get_address_point(x['bin'], geom_col = 'the_geom'), axis = 1).values

TypeError: 'int' object is not iterable

In [24]:
get_address_point('1027194', geom_col = 'the_geom')

(  address_id h_no                       the_geom bin hyphen_typ side_of_st  \
 0    5161024  512  POINT (982905.309 211717.955)   0          N          1   
 
   borocode zipcode                   created                  modified  \
 0        1   10011  2014-07-18T00:00:00.000Z  2020-04-01T00:00:00.000Z   
 
   st_name physicalid pre_direct post_type full_stree  
 0      23     129100          W        ST   W  23 ST  ,
 ['1', '2', '7', '1', '9', '4'])

In [9]:
structs = b[~pd.isnull(b['bin'])]

In [10]:
final_df = master_geosupport_func2(structs)

In [11]:
final_df.columns

Index(['bin', 'high_address_number', 'low_address_number', 'street_name',
       'out_boro_name1', 'address_id', 'b7sc', 'tpad_bin_status', 'type',
       'type_meaning', 'out_bbl', 'out_TPAD_bin', 'out_TPAD_bin_status',
       'out_TPAD_conflict_flag', 'out_error_message', 'out_grc', 'in_bin',
       'addressable', 'out_zip_code', 'out_hurricane_zone', 'out_co', 'out_cd',
       'out_sd', 'out_nta', 'out_ad', 'out_com_dist', 'out_fire_bat',
       'out_fire_co', 'out_fire_co_str', 'out_fire_div', 'out_b10sc1',
       'out_police_patrol_boro', 'out_police_area', 'out_police_pct',
       'out_san_sched', 'out_san_dist_section', 'out_san_recycle',
       'out_san_reg', 'out_san_org_pick_up', 'out_usps_city_name',
       'out_preferred_lgc', 'out_sos_ind', 'out_physical_id'],
      dtype='object')

In [14]:
final_df[['high_address_number', 'low_address_number', 'street_name', 'out_boro_name1', 'out_zip_code', 'bin', 'address_id']]

,high_address_number,low_address_number,street_name,out_boro_name1,out_zip_code,bin,address_id
0,9000,9000,BAY PARKWAY,BROOKLYN,11214,3340691,3217154
1,694,694,THOMAS S BOYLAND STREET,BROOKLYN,11212,3081931,3109361
2,512,512,WEST 23 STREET,MANHATTAN,10011,0,5161024
3,81,81,FRANKLIN STREET,BROOKLYN,11222,3400033,5168623
4,2134,2134,CROPSEY AVENUE,BROOKLYN,11214,3170079,3217153
5,9000,8922,BAY PARKWAY,BROOKLYN,NaN,3340691,NaN
6,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,,3423964,NaN
7,,,BETSY HEAD POOL,BROOKLYN,11212,3392494,NaN
8,2133,2133,STRAUSS STREET,BROOKLYN,11212,3392494,NaN
9,,,BETSY HEAD PLAY CENTER,BROOKLYN,11212,3081931,NaN


In [14]:
#Replace the nan values for address point only address with an out_grc = -99
final_df['out_grc'] = final_df['out_grc'].fillna('-99')

In [15]:
#Convert out_grc to string (as it should be) to avoid errors in larger data
final_df['out_grc'] = final_df.apply(lambda x: str(x['out_grc']), axis = 1)

In [16]:
def add_ck(df):
    #Records with 00 or -99 GRC codes are valid to check for "addressable" condition required for Function 1B
    if df['out_grc'] in ['00', '-99']:
            
            #Check for equality and validity of low and high house number. The low and high house numbers need cannot be blank and
            #must be equal to use as input into function 1.
            if (df['high_address_number'] == df['low_address_number'] and 
               (df['high_address_number'] != '' and df['low_address_number'] != '')):          
                add_val = 'Addressable'
                
            #If the low and high range are not equal then the record should not be input into function 1.
            else:
                add_val = 'Non-Addressable, Range'
                

    else:
        add_val = 'Non-Addressable: {}'.format(df['out_error_message'])
        
    return add_val

In [18]:
final_df['addressable'] = final_df.apply(lambda x: add_ck(x), axis = 1)

In [114]:
func1B_outputs = (final_df[final_df['addressable'] == 'Addressable']
                  .apply(lambda row: func1b(borough= row['out_boro_name1'], 
                                            addressno= row['high_address_number'], 
                                            streetname= row['street_name'],
                                            api_key=geo_key,
                                            ip=geo_ip),axis =1))

In [116]:
strip_vals(func1B_outputs)

In [106]:
func1B_out_df = (pd.DataFrame(list(func1B_outputs), index = list(func1B_outputs.index))
                 .rename(columns = {'in_hnd': 'high_address_number',
                                    'in_stname1': 'street_name'})
                 .assign(addressable = 'Addressable'))

In [119]:
func1B_out_df = (pd.DataFrame(list(func1B_outputs), index = list(func1B_outputs.index)))            

In [109]:
final = final_df.join(func1B_out_df)

TypeError: join() got an unexpected keyword argument 'suffix'

In [41]:
final = final_df.merge(func1B_out_df, 
                      how = 'left',`
                      on = ['high_address_number', 'street_name', 'out_boro_name1', 'addressable'])

In [43]:
func1B_out_df

,out_zip_code,out_hurricane_zone,out_co,out_cd,out_sd,out_nta,out_ad,out_com_dist,out_fire_bat,out_fire_co,...,out_san_reg,out_san_org_pick_up,out_usps_city_name,out_preferred_lgc,out_sos_ind,out_physical_id,street_name,high_address_number,out_boro_name1,addressable
0,11214,1,43,11,23,BK27 / Bath Beach,46,311,43,Ladder 166,...,,,BROOKLYN,31733001,Address is on the right when facing from BELT ...,0051350,BAY PARKWAY,9000,BROOKLYN,Addressable
1,,,,,,/,,,,,...,,,,,,,NAN,694,,Addressable
2,,,,,,/,,,,,...,,,,,,,NAN,512,,Addressable
3,11222,3,33,12,26,BK76 / Greenpoint,50,301,28,Ladder 106,...,MWF,,BROOKLYN,34203001,Address is on the left when facing from NOBLE ...,0064739,FRANKLIN STREET,81,BROOKLYN,Addressable
4,11214,3,43,11,23,BK27 / Bath Beach,47,311,42,Engine 243,...,WS,,BROOKLYN,33063001,Address is on the right when facing from BAY ...,0042457,CROPSEY AVENUE,2134,BROOKLYN,Addressable
5,11212,6,41,09,19,BK81 / Brownsville,55,316,44,Engine 231,...,MWF,,BROOKLYN,38243001,Address is on the left when facing from DUMONT...,0072909,STRAUSS STREET,2133,BROOKLYN,Addressable
6,11212,6,41,09,19,BK81 / Brownsville,55,316,44,Engine 231,...,MWF,,BROOKLYN,33433001,Address is on the right when facing from AMBOY...,0072967,DUMONT AVENUE,202,BROOKLYN,Addressable
7,11212,6,41,09,19,BK81 / Brownsville,55,316,44,Engine 231,...,MWF,,BROOKLYN,35663001,Address is on the left when facing from AMBOY ...,0072290,LIVONIA AVENUE,167,BROOKLYN,Addressable


In [42]:
final.head(15)

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,b7sc,tpad_bin_status,type,type_meaning,...,out_police_pct,out_san_sched,out_san_dist_section,out_san_recycle,out_san_reg,out_san_org_pick_up,out_usps_city_name,out_preferred_lgc,out_sos_ind,out_physical_id
0,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,NaN,NaN,NaN,NaN,...,62,,311 / 11,,,,BROOKLYN,31733001,Address is on the right when facing from BELT ...,0051350
1,3081931,694,694,NaN,NaN,3109361,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,512,512,NaN,NaN,5161024,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,34203001,N/A,,Ordinary Address Range,...,94,1A,301 / 011,EM,MWF,,BROOKLYN,34203001,Address is on the left when facing from NOBLE ...,0064739
4,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,33063001,N/A,,Ordinary Address Range,...,62,5E,311 / 115,ES,WS,,BROOKLYN,33063001,Address is on the right when facing from BAY ...,0042457
5,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,31733001,N/A,,Ordinary Address Range,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,31049501,N/A,B,Non-Addressable Unnamed Building,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3392494,,,BETSY HEAD POOL,BROOKLYN,NaN,31207503,N/A,X,Constituent NAP of Complex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3392494,2133,2133,STRAUSS STREET,BROOKLYN,NaN,38243001,N/A,,Ordinary Address Range,...,73,2A,316 / 162,EF,MWF,,BROOKLYN,38243001,Address is on the left when facing from DUMONT...,0072909
9,3081931,,,BETSY HEAD PLAY CENTER,BROOKLYN,NaN,31207502,N/A,X,Constituent NAP of Complex,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Start Testing 1/13

In [7]:
structs_df = b[~pd.isnull(b['bin'])]

In [9]:
funcap_outputs = structs_df.apply(lambda row: get_address_point2(row['bin'],
                                                                 geom_col = 'the_geom'), axis =1)

In [10]:
#Convert the address points results to a geodataframe, defining the initial projection as WGS84(epsg:4326)
aps_gdf = (gpd.GeoDataFrame(list(funcap_outputs), 
                            geometry = 'the_geom',
                            crs = 'epsg:4326').fillna(np.nan))

In [11]:
#Convert the CRS to the NYC Standard of State Plane New York Long Island (epsg:2263)
aps = (aps_gdf[~pd.isnull(aps_gdf['the_geom'])]
       .to_crs('epsg:2263')
       .copy())

In [12]:
#Call function 1N to get the normalized street names
#In 1N we want to keep out_boro_name1 out_st_name1
func1N_outputs = aps.apply(lambda row: func1n(borough=row['borocode'],
                                              streetname = row['full_stree'],
                                              api_key = geo_key,
                                              ip = geo_ip),axis =1)


In [13]:
#Strip white space from all returned values because it's not necessary
strip_vals(func1N_outputs)

In [14]:
#Convert the results of 1N to a dataframe. The outputs need to be converted to a list
#and the indexes as well so the correct index applies to the correct row.
func1N_out_df = pd.DataFrame(list(func1N_outputs), list(func1N_outputs.index))

In [15]:
#Join the 1N results to the address points dataframe based on the index
aps_1N_df = aps.join(func1N_out_df)

In [16]:
#Rename the h_no column to high_address_number and low_address_number, this is needed for later deduplication
aps_1N_df['high_address_number'] = aps_1N_df['h_no']
aps_1N_df['low_address_number'] = aps_1N_df['h_no']

#Add a column to signify this is a posted addresses
aps_1N_df['posted_address'] = 1

In [17]:
#Rename the streetname column from out_stname1
aps_1N_df.rename(columns = {'out_stname1':'street_name'},inplace = True)

In [18]:
#Call function BN to get addresses for BINs:
funcBN_outputs = structs_df.apply(lambda row: funcbn(bn = row['bin'],
                                                     out_keys = out_keys,
                                                     grc_err = grc_err,
                                                     api_key = geo_key,
                                                     ip = geo_ip), axis =1)

In [19]:
#Flatten the list of BINs returned
funcBN_outputs_flat = flat_list(funcBN_outputs)

In [20]:
#Strip white space from all returned values because it's not necessary
strip_vals(funcBN_outputs_flat)

In [21]:
#Convert the results of BN to a dataframe
BN_out_df = pd.DataFrame(funcBN_outputs_flat)

In [22]:
#Add column to signify the official addresses
BN_out_df['official_address'] = 1

In [65]:
BN_out_df.columns

Index(['b7sc', 'bin', 'high_address_number', 'low_address_number',
       'street_name', 'tpad_bin_status', 'type', 'type_meaning', 'out_bbl',
       'out_TPAD_bin', 'out_TPAD_bin_status', 'out_TPAD_conflict_flag',
       'out_error_message', 'out_grc', 'in_bin', 'out_boro_name1',
       'official_address'],
      dtype='object')

In [67]:
 BN_out_df['official_address'] = BN_out_df.apply(lambda x: official_address(x), axis = 1)

In [68]:
BN_out_df

,b7sc,bin,high_address_number,low_address_number,street_name,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,in_bin,out_boro_name1,official_address
0,34203001,3400033,81,81,FRANKLIN STREET,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3400033,BROOKLYN,1
1,33063001,3170079,2134,2134,CROPSEY AVENUE,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3170079,BROOKLYN,1
2,31733001,3340691,9000,8922,BAY PARKWAY,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3340691,BROOKLYN,1
3,31049501,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3423964,BROOKLYN,1
4,31207503,3392494,,,BETSY HEAD POOL,N/A,X,Constituent NAP of Complex,3035700001,N/A,N/A,N/A,,00,3392494,BROOKLYN,1
5,38243001,3392494,2133,2133,STRAUSS STREET,N/A,,Ordinary Address Range,3035700001,N/A,N/A,N/A,,00,3392494,BROOKLYN,1
6,31207502,3081931,,,BETSY HEAD PLAY CENTER,N/A,X,Constituent NAP of Complex,3035700001,N/A,N/A,N/A,,00,3081931,BROOKLYN,1
7,33433001,3081931,202,202,DUMONT AVENUE,N/A,,Ordinary Address Range,3035700001,N/A,N/A,N/A,,00,3081931,BROOKLYN,1
8,35083001,3081931,752,694,THOMAS S BOYLAND STREET,N/A,,Ordinary Address Range,3035700001,N/A,N/A,N/A,,00,3081931,BROOKLYN,1
9,35663001,3081931,167,167,LIVONIA AVENUE,N/A,,Ordinary Address Range,3035700001,N/A,N/A,N/A,,00,3081931,BROOKLYN,1


In [25]:
aps_1N_df

,address_id,h_no,the_geom,bin,hyphen_typ,side_of_st,borocode,zipcode,created,st_name,...,post_type,full_stree,modified,pre_direct,out_boro_name1,out_grc,street_name,high_address_number,low_address_number,posted_address
0,5168623,81,POINT (995998.149 204776.450),3400033,N,1,3,11222,2015-12-15T00:00:00.000Z,FRANKLIN,...,ST,FRANKLIN ST,NaN,NaN,BROOKLYN,00,FRANKLIN STREET,81,81,1
1,3217153,2134,POINT (984261.245 157105.052),3170079,N,2,3,11214,2009-02-13T00:00:00.000Z,CROPSEY,...,AVE,CROPSEY AVE,NaN,NaN,BROOKLYN,00,CROPSEY AVENUE,2134,2134,1
2,3217154,9000,POINT (983853.114 156079.787),3340691,N,2,3,11214,2009-02-13T00:00:00.000Z,BAY,...,PKWY,BAY PKWY,2018-07-26T00:00:00.000Z,NaN,BROOKLYN,00,BAY PARKWAY,9000,9000,1
5,3109361,694,POINT (1008624.871 180844.155),3081931,N,2,3,11212,2009-02-13T00:00:00.000Z,THOMAS S BOYLAND,...,ST,THOMAS S BOYLAND ST,2013-07-15T00:00:00.000Z,NaN,BROOKLYN,00,THOMAS S BOYLAND STREET,694,694,1
6,5161024,512,POINT (982905.309 211717.955),0,N,1,1,10011,2014-07-18T00:00:00.000Z,23,...,ST,W 23 ST,2020-04-01T00:00:00.000Z,W,MANHATTAN,00,WEST 23 STREET,512,512,1


In [24]:
BN_out_df

,b7sc,bin,high_address_number,low_address_number,street_name,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,in_bin,out_boro_name1,official_address
0,34203001,3400033,81,81,FRANKLIN STREET,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3400033,BROOKLYN,1
1,33063001,3170079,2134,2134,CROPSEY AVENUE,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3170079,BROOKLYN,1
2,31733001,3340691,9000,8922,BAY PARKWAY,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3340691,BROOKLYN,1
3,31049501,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3423964,BROOKLYN,1
4,31207503,3392494,,,BETSY HEAD POOL,N/A,X,Constituent NAP of Complex,3035700001,N/A,N/A,N/A,,00,3392494,BROOKLYN,1
5,38243001,3392494,2133,2133,STRAUSS STREET,N/A,,Ordinary Address Range,3035700001,N/A,N/A,N/A,,00,3392494,BROOKLYN,1
6,31207502,3081931,,,BETSY HEAD PLAY CENTER,N/A,X,Constituent NAP of Complex,3035700001,N/A,N/A,N/A,,00,3081931,BROOKLYN,1
7,33433001,3081931,202,202,DUMONT AVENUE,N/A,,Ordinary Address Range,3035700001,N/A,N/A,N/A,,00,3081931,BROOKLYN,1
8,35083001,3081931,752,694,THOMAS S BOYLAND STREET,N/A,,Ordinary Address Range,3035700001,N/A,N/A,N/A,,00,3081931,BROOKLYN,1
9,35663001,3081931,167,167,LIVONIA AVENUE,N/A,,Ordinary Address Range,3035700001,N/A,N/A,N/A,,00,3081931,BROOKLYN,1


In [26]:
##Combine output of 1N and BN and deduplicate those records based on Borough, High House Number, Low House Number, Street Name and Hyphen Type

#Define the columns to match for deduplication
cols_to_match = ['bin','high_address_number','low_address_number','street_name','out_boro_name1']

In [27]:
#Keep address_id and the cols_to_match from address points and 1N output
df1 = aps_1N_df[cols_to_match+['address_id', 'posted_address']]

In [28]:
df2 = BN_out_df

In [29]:
combined_deduped = pd.concat([df1,df2]).drop_duplicates(cols_to_match,keep='last')

In [30]:
combined = pd.concat([df1,df2])

In [31]:
combined

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,posted_address,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,in_bin,official_address
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3081931,694,694,THOMAS S BOYLAND STREET,BROOKLYN,3109361,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,512,512,WEST 23 STREET,MANHATTAN,5161024,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,NaN,NaN,34203001,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3400033,1.0
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,NaN,NaN,33063001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3170079,1.0
2,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,NaN,31733001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3340691,1.0
3,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,NaN,31049501,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3423964,1.0
4,3392494,,,BETSY HEAD POOL,BROOKLYN,NaN,NaN,31207503,N/A,X,Constituent NAP of Complex,3035700001,N/A,N/A,N/A,,00,3392494,1.0


In [32]:
df_w_ap_ids = pd.concat(g for _, 
    g in combined.groupby(cols_to_match) if len(g) > 1).drop_duplicates(cols_to_match,keep='first')

In [34]:
df_w_ap_ids

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,posted_address,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,in_bin,official_address
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df_new = combined_deduped.merge(df_w_ap_ids,how = 'left',suffixes=('', '_y'), 
                   left_on = cols_to_match, right_on=cols_to_match)

In [55]:
df_new.address_id.fillna(df_new.address_id_y, inplace=True)

In [57]:
df_new.posted_address.fillna(df_new.posted_address_y, inplace=True)

In [58]:
df_new.official_address.fillna(df_new.official_address_y, inplace=True)

In [59]:
df_new.drop(df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [60]:
df_new

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,posted_address,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,in_bin,official_address
0,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3081931,694,694,THOMAS S BOYLAND STREET,BROOKLYN,3109361,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,512,512,WEST 23 STREET,MANHATTAN,5161024,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,1.0,34203001,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3400033,1.0
4,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,1.0,33063001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3170079,1.0
5,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,NaN,31733001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3340691,1.0
6,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,NaN,31049501,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3423964,1.0
7,3392494,,,BETSY HEAD POOL,BROOKLYN,NaN,NaN,31207503,N/A,X,Constituent NAP of Complex,3035700001,N/A,N/A,N/A,,00,3392494,1.0
8,3392494,2133,2133,STRAUSS STREET,BROOKLYN,NaN,NaN,38243001,N/A,,Ordinary Address Range,3035700001,N/A,N/A,N/A,,00,3392494,1.0
9,3081931,,,BETSY HEAD PLAY CENTER,BROOKLYN,NaN,NaN,31207502,N/A,X,Constituent NAP of Complex,3035700001,N/A,N/A,N/A,,00,3081931,1.0


### End Testing 1/13

# Dan, you can ignore cells below this one for now

In [10]:
# structs.to_csv(r'structs_all.csv')

In [11]:
# structs.to_csv(r'structs_top5.csv')

In [42]:
bins = list(structs.bin.values)

In [13]:
# bins = list(b[b['bin'].notnull()]['bin'])

In [14]:
# bins

In [8]:
structs

,fid,objectid,parks_id,bin,bbl,doitt_id,ground_elevation,height_roof,construction_year,alteration_year,demolition_year,n_doitt_ids,doitt_source,overlap_except,unexpected_change,parks_overlap,shape,row_hash
0,1384,5139,B001-BLG0989,3400033.0,3025650014,1157878.0,15.0,14.910000,1940.0,None,None,1,current,True,False,False,b'\xd7\x08\x00\x00\x01\x04\x05\x00\x00\x00\x00...,b'v+\xdd\x9ek\x9e\xc3Jq\x0cP[\xeeRm$\xe2\x1d\x...
2,1755,6494,B007-BLG1117,3170079.0,3064900024,609048.0,21.0,15.060000,1900.0,None,None,1,current,True,False,False,"b""\xd7\x08\x00\x00\x01\x04\t\x00\x00\x00\x80\x...",b'^\xe8f\x8c\x9a\x8c\xe8l\x91;\x16\x9aP@\x8f\x...
3,2449,6185,B007-BLG1118,3340691.0,3064900024,921467.0,10.0,21.090000,1900.0,None,None,1,current,True,False,False,b'\xd7\x08\x00\x00\x01\x04\n\x00\x00\x00\x80\x...,"b""\xa4X\xc3OO\xd0\x07\x00\xf3\xbd\xb6\x002\xfb..."
4,667,6184,B007-BLG1119,3423964.0,3064900024,1272109.0,13.0,12.049366,NaN,None,None,1,current,True,False,False,b'\xd7\x08\x00\x00\x01\x04\x07\x00\x00\x00\x00...,b'\x14x\xa4\x9c\x9fR8\xfc\x7f\x80aXP\xeb\xfb\x...


In [29]:
# %%timeit
get_ap_output = structs.apply(lambda row: get_address_point2(row['bin'],
              geom_col = 'the_geom'),axis =1)

In [9]:
aps_gdf = gpd.GeoDataFrame(flat_list(get_ap_output), geometry = 'the_geom',crs = 'epsg:4326').fillna(np.nan)
aps_gdf.head()

NameError: name 'get_ap_output' is not defined

In [38]:
aps = aps_gdf[~pd.isnull(aps_gdf['the_geom'])].to_crs('epsg:2263').copy()
aps.head()

RuntimeError: b'no arguments in initialization list'

In [19]:
# get_ap_output.values

In [20]:
# try_flat1 = flat_list(get_ap_output)#.loc[0]#.keys()
# try_flat1

In [21]:
# pd.DataFrame(try_flat1)

In [22]:
# aps, failed_bins = get_address_point(bins, geom_col = 'the_geom')

In [23]:
# type(aps)

In [24]:
# aps.head()

In [25]:
# failed_bins

In [26]:
# aps.bin.values

In [27]:
grc_err = ['01/F', '20', '21', '22', '23']
out_keys = [
    'AddressRangeList', 'out_bbl', 'out_TPAD_bin', 'out_TPAD_bin_status',
    'out_TPAD_conflict_flag', 'out_error_message', 'out_grc',
    'out_sanborn_boro', 'in_bin', 'out_boro_name1']

In [28]:
# for abin in bins
# bn = funcbn(bn = bins[2], out_keys = out_keys, grc_err = grc_err, api_key = geo_key, ip = geo_ip)

In [29]:
# structs['bin']

In [77]:
funcBN_outputs = structs.apply(lambda row: funcbn(row['bin'],
              out_keys=out_keys,
              grc_err=grc_err,
              api_key=geo_key,
              ip=geo_ip),axis =1)

In [31]:
# funcBN_outputs

In [78]:
t = flat_list(funcBN_outputs)
# t

In [79]:
strip_vals(t)

In [34]:
# t

In [80]:
BN_out_df = pd.DataFrame(t)
BN_out_df.head()

,b7sc,bin,high_address_number,low_address_number,street_name,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin,out_boro_name1
0,34203001,3400033,81,81,FRANKLIN STREET,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033,BROOKLYN
1,33063001,3170079,2134,2134,CROPSEY AVENUE,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079,BROOKLYN
2,31733001,3340691,9000,8922,BAY PARKWAY,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691,BROOKLYN
3,31049501,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964,BROOKLYN


In [36]:
# pd.DataFrame(strip_vals(flat_list(funcBN_outputs.values)))

In [37]:
# bn

In [38]:
# a = aps.iloc[0][['borocode', 'full_stree']]

In [39]:
# aps.values

In [81]:
func1N_outputs = aps.apply(lambda row: func1n(borough=row['borocode'],
                             streetname=row['full_stree'],
                             api_key=geo_key,
                             ip=geo_ip),axis =1)

In [82]:
list_1N = flat_list(func1N_outputs)

In [42]:
# list_1N

In [83]:
strip_vals(list_1N)

In [44]:
# list_1N

In [84]:
func1N_out_df = pd.DataFrame(list_1N)

In [85]:
aps_1N_df = aps.join(func1N_out_df)

In [47]:
# aps

In [48]:
# func1N_out_df

In [86]:
BN_out_df['func_name'] = 'BN'

In [87]:
aps_1N_df['func_name'] = '1N'

In [51]:
# func1N_out_df['func_name'] = '1N'

In [52]:
# aps_1N_df.columns.values

In [53]:
# BN_out_df.columns.values

In [54]:
# BN_out_df[['bin','high_address_number','low_address_number','street_name','out_boro_name1']]

In [88]:
aps_1N_df['high_address_number'] = aps_1N_df['h_no']

In [89]:

aps_1N_df['low_address_number'] = aps_1N_df['h_no']

In [90]:
aps_1N_df.rename(columns = {'out_stname1':'street_name'},inplace = True)

In [91]:
cols_to_match = ['bin','high_address_number','low_address_number','street_name','out_boro_name1']

In [92]:
# cols_to_match+['address_id','func_name']

In [60]:
# aps_1N_df

In [117]:
df1 = aps_1N_df[cols_to_match+['address_id','func_name']]
df1

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,1N
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,1N
2,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N


In [118]:
df2 = BN_out_df#[cols_to_match+['func_name','type_meaning']]
df2

,b7sc,bin,high_address_number,low_address_number,street_name,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin,out_boro_name1,func_name
0,34203001,3400033,81,81,FRANKLIN STREET,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033,BROOKLYN,BN
1,33063001,3170079,2134,2134,CROPSEY AVENUE,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079,BROOKLYN,BN
2,31733001,3340691,9000,8922,BAY PARKWAY,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691,BROOKLYN,BN
3,31049501,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964,BROOKLYN,BN


In [119]:
combined_deduped = pd.concat([df1,df2]).drop_duplicates(cols_to_match,keep='last')
combined_deduped

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin
2,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,NaN,BN,34203001,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,NaN,BN,33063001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079
2,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,BN,31733001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691
3,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,BN,31049501,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964


In [120]:
combined = pd.concat([df1,df2])
combined

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,NaN,BN,34203001,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,NaN,BN,33063001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079
2,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,BN,31733001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691
3,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,BN,31049501,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964


In [121]:
df_w_ap_ids = pd.concat(g for _, g in combined.groupby(cols_to_match) if len(g) > 1).drop_duplicates(cols_to_match,keep='first')

In [122]:
df_new = combined_deduped.merge(df_w_ap_ids,how = 'left',suffixes=('', '_y'), 
                       left_on = cols_to_match, right_on=cols_to_match)
# df_new.drop(df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [123]:
df_new.address_id.fillna(df_new.address_id_y, inplace=True)

In [124]:
df_new.drop(df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
df_new

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin
0,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,BN,34203001,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033
2,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,BN,33063001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079
3,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,BN,31733001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691
4,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,BN,31049501,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964


In [69]:
combined_deduped.groupby('bin').count()[]

,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name
bin,,,,,,
0.0,0,0,0,1,0,1
1000000.0,0,0,0,1,0,1
1003210.0,0,0,0,1,0,1
1088300.0,0,0,0,1,0,1
1090070.0,0,0,0,1,0,1
...,...,...,...,...,...,...
5169770,2,2,1,1,1,2
5169802,1,1,1,1,0,1
5169908,2,2,2,2,0,2


In [70]:
combined_deduped[combined_deduped['high_address_number']==combined_deduped['low_address_number']]

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,1N
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,1N
2,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N
5,3081931,694,694,WASHINGTON AVENUE,BROOKLYN,3109361,1N
6,0,512,512,WASHINGTON AVENUE,BROOKLYN,5161024,1N
...,...,...,...,...,...,...,...
3737,2086438,,,5 BOROUGH SHOPS,BRONX,NaN,BN
3738,2086438,850,850,EAST 138 STREET,BRONX,NaN,BN
3739,2086438,231,231,WALNUT AVENUE,BRONX,NaN,BN
3740,2086438,,,EAST 137 STREET,BRONX,NaN,BN


In [64]:
# func1n(borough = a['borocode'], streetname = a['full_stree'], api_key = geo_key, ip = geo_ip)

In [65]:
# df = master_geosupport_func(bins)

In [66]:
# df

In [67]:
# failed_bins

In [68]:
# aps